In [14]:
import sentence_transformers
print(sentence_transformers.__version__)

c:\Users\nilay\OneDrive - Cal State Fullerton (1)\Desktop\NILAY-TO-JOB-DATA\SPRING 2024\Projects\Gmail-JobEmail-Classifier\myvenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


3.2.1


In [15]:
import numpy as np
print(np.__version__)

2.1.3


In [17]:
import matplotlib

print(matplotlib.__version__)

3.9.2


In [18]:
import sklearn
print(sklearn.__version__)

1.5.2
